# Zusammenfassung

Der letzte Abschnitt baute auf die Grundlagen des vierten Abschnitts auf und vollendet den Themenbereich App-Entwicklung.  
In Android besitzt jede Aktivität und jedes Fragment einen eigenen **Lebenszyklus**. Dieser beschreibt den aktuellen Status des Elements und kann als Zustandsdiagramm dargestellt werden. Bei einer Änderung des Zustandes werden bestimmte Methoden aufgerufen, die überschrieben werden können. Diese Zustandsänderung wird von den Benutzer:innen herbeigeführt, indem zum Beispiel die App gestartet oder beendet wird. Jedem Event ist eine Methode zugeordnet, die dann ausgeführt wird. Sollen zu diesen Zeitpunkten bestimmte Operationen ausgeführt werden, können diese überschrieben werden. Eine Übersicht ist in der folgenden Abbildung dargestellt:

![Lebenszyklus](images/Lifecycle.png) 

In der Abbildung wird deutlich, dass diese Zeitpunkte immer in Paaren auftreten. Wird zum Beispiel eine Aktivität gestartet, wird `onCreate()` aufgerufen. Der Gegensatz dazu ist die Methode `onDestroy()`, die den Code beinhaltet, der beim Schließen der Aktivität ausgeführt wird. Dort sollen alle Aktionen beendet werden, die in `onCreate()` angestoßen wurden.  
Besonders hervorgehoben soll noch die linke Hälfte der Abbildung werden. Wenn zu viele Apps im Hintergrund geöffnet sind, kann die Rechenleistung für die im Vordergrund befindliche App eingeschränkt werden. Dann schließt das Betriebssystem automatisch Apps im Hintergrund. Diese werden jedoch weiterhin als geöffnet angezeigt. Wird die App wieder geöffnet und *fortgesetzt*, wird wieder `onCreate()` aufgerufen.

Um den Überblick bei größeren und komplexeren Apps nicht zu verlieren, ist eine strukturierte **Architektur** notwendig. Google empfiehlt die Auftrennung der Funktionen in verschiedene Teilbereiche:  
- In Aktivitäten und Fragmenten werden die Daten dargestellt und Aktionen verarbeitet
- Die Verarbeitung und Kalkulation der Daten erfolgt in einem *ViewModel*
- Datenbak- und Netzwerkzugriffe werden über *Repositoryies* verwaltet

Das **ViewModel** ist die Klasse, in der die Daten verarbeitet werden. In diesem wird auch auf die *Repositories* zugegriffen. Werden in der Aktivität oder im Fragment Aktionen ausgelöst, werden diese an das *ViewModel* weitergeleitet. Erstellt wird ein Objekt der Klasse mit Hilfe des Entwurfsmusters *Factory*. Dadurch können diesem einfach Parameter, wie beispielsweise den Kontext der Datenbanken, übergeben werden. Nebenbei wird es auch mit dem *Lifecycle* des zugehörigen Elements verbunden. In der *View* wird dann zuerst ein Objekt der Fabrik erzeugt, mit dem das *ViewModel* erstellt wird. Dadurch ist der Zugriff von der Ansicht auf das Model möglich.  
Die entgegengesetzte Richtung vom Model zur Ansicht ist aber nicht direkt navigierbar. Dies ist durch einen Umweg mit **LiveData** möglich. Diese Klasse, die als Mantel für einen Wert, der im Feld `value` gespeichert ist, dient, verwendet das Entwurfsmuster *Observer* und kann somit zum Beispiel von einem Fragment mit der Methode `observe` beobachtet werden. Ändert sich der *nullable* Wert, wird eine definierte Aktion, die in einem Lambda-Ausdruck angegeben werden muss, ausgeführt. Jedoch wird auch hier zwischen veränderbaren und unveränderbaren *LiveData* unterschieden. Während bei der normalen Klasse der Wert nicht verändert werden kann, ist dies bei *MutableLiveData* beliebig möglich.

Bei der Verwendung vieler verschiedenen Funktionen, wie zum Beispiel einer Datenbank oder einem Netzwerkzugriff, ist es unumgänglich, dass **verschiedene Modelle** für das gleiche Datenstück benötigt werden. Dies hängt vor Allem an den unterschiedlichen Anforderungen, die an die Datenklassen gestellt werden, hilft aber auch bei der Übersichtlichkeit. Die Benennung kann eigenständig gewählt werden. Ein Vorschlag:  

![Modelle](images/Model.png) 

Das *DomainModel* beschreibt die *normale* Repräsentation, wie sie in Aktivitäten, Fragmenten und *ViewModels* verwendet wird. Eine *DatabaseEntity* ist wie der Name schon vermuten lässt, ein Eintrag in einer Datenbank. Hier ist beispielsweise die Verwendung eines primären, einzigartigen Schlüssels notwendig, der im Model nicht benötigt wird. Daten aus dem Internet besitzen das Suffix *Property* und hängen von der Rückgabe der Anfrage ab. Verbunden sind die verschiedenen Datenklassen mit Erweiterungsmethoden, die die Modelle untereinander konvertieren. Die Umwandlung findet größtenteils in den *Repositories* statt.

**Koroutinen** sind in Kotlin die neuen, besseren Threads. Mit ihnen kann Code parallel ausgeführt werden. In diesem Kontext werden folgende Fachbegriffe wichtig:
- *Threads* werden von der Java Virtual Machine (JVM) verwaltet und verwenden die Multi-Threading-Funktionen des Betriebssystems. Sie besitzen zudem einen eigenen Stack-Speicher. Gemeinsame Informationen werden über einen Heap-Speicher mit anderen Threads geteilt.
- *Koroutinen* werden von Kotlin verwaltet und sind nicht an einen konkreten CPU gebunden, sondern können beliebig verschoben werden. Beispielsweise wird eine Koroutine zuerst auf Kern 1 gestartet und dann auf dem Dritten beendet. Sie sind deutlich flexibler als Threads. Zudem besitzen sie keinen eigenen Speicher und sind ressourcensparsamer. Im Hintergund werden sie auf einer geringen Anzahl an Threads ausgeführt.
- Ein *Scope* ist der Kontext einer Koroutine. *GlobalScope* ist beispielsweise die *Top-Level*-Ebene.
- Der *Dispatcher* ist mit dem *Scope* verbunden und entscheidet von welchem Thread die Koroutine ausgeführt wird. Außerdem lässt sich aus diesem der Aufgabenbereich ableiten. Mögliche *Dispatcher* sind: Default, IO, Unconfined oder eigens definierte mit *newSingleThreadContext("name")*.


Gestartet werden kann eine Koroutine standardmäßig mit `launch` auf einem *Scope*. Die Anweisungen werden in einem folgenden Lambda-Ausdruck definiert. In diesem Fall findet eine parallele Berechnung des Hauptthreads und der Koroutine statt. Wird *launch* hingegen in einem Lambda-Ausdruck von `runBlocking` gestartet, wird der Hauptthread bis zur Beendigung der Koroutinen gestoppt. Mit `join()` kann auf eine *Coroutine* gewartet werden, wenn diese vorher einer Variable zugewiesen wurde. Soll ein Ergebnis zurückgegeben werden, findet statt *launch* `async` Anwendung. Die Rückgabe ist ein Objekt der Klasse *Deferred&#60;T>*, wobei *T* der eigentliche Datentyp der Rückgabe ist. Auf das Ergebnis kann entweder mit `getComplete()`, was bei einer immer noch laufenden Prozess einen Fehler wirft, oder `await()` zugegriffen werden. Das Stoppen einer Ausführung ist mit `cancel()` möglich. Der auszuführende Code muss immer in dem folgenden Lambda-Ausdruck lokalisiert sein. In diesem können aber auch Funktionen ausgerufen werden. Voraussetzung ist, dass sie das Schlüsselwort `suspend` besitzen.
    
Verwendet werden die Koroutinen, in Verbindung mit einem **Repository**. Dieses verwaltet die Zugriffe auf Datenbanken und Internetabfragen. Auch werden in diesem die verschiedenen Repräsentationen der Daten ineinander konvertiert und so für jeden Teil der App immer das richtige Modell vorhanden ist. In dieser Klasse gibt es meistens zwei verschiedene Arten an Funktionen. Einerseits Methoden, die Daten aus dem Internet abfragen und diese dann in einer Datenbank speichern oder Felder, die Einträge einer Datenbank beinhalten. Die *ViewModels* greifen dann auf die Felder zu und verwenden Sie für ihre Berechnungen oder geben sie an die Ansichten weiter.

Eine beliebte Freigabe ist die auf den **Standort** des Geräts. Dieser kann mit einem *LocationManager*, der bereits in Android als Systemdienst vorhanden ist, erlangt werden. Abgesehen von einer einmaligen Abfrage ist auch eine zeitliche oder örtliche Wiederholung nach dem Zurücklegen einer definierten Distanz mit *requestLocationUpdates* möglich. Zuletzt muss dem methodenaufruf ein *LocationListener* mit einem Lambda-Ausdruck übergeben werden, in dem festgelegt ist, wie nach einer Abfrage verfahren werden soll. Vor dem Aufruf muss jedoch die Erlaubnis abgefragt und wenn noch diese noch nicht gegeben wurde, eingeholt werden.

Der **Internetzugriff** ist mit Hilfe der Bibliothek *retrofit* möglich. Der Aufbau ähnelt dem einer *Room*-Datenbank. Das Herzstück stellt wieder eine Schnittstelle dar. In dieser können Methoden deklariert werden, die mit Hilfe von Annotationen Daten einer Api (*Application Programming Interface*) abrufen können. Die Verbreitetsten sind *GET* und *POST*. Zusätzlich muss bei ihnen der genaue Endpunkt von der Basis-Url aus angegeben werden. Parameter können den Aufrufen als Parameter der Methode hinzugefügt werden. Diesen wird die Annotation `@Query` mit dem Namen des Parameters vorangestellt. Das Ergebnis der Anfrage kann mit verschiedenen Konvertierern in Objekte umgewandelt werden. Für JSON-Dateien ist beispielweise die Bibliothek *moshi* mit seinem Konvertierer zu empfehlen. Dabei wird das Resultat als Datenklasse nachgebaut und die Annotation `@JsonClass(generateAdapter = true)` vorangestellt. Bei Aufruf der Methode kann dann ein Objekt der Datenklasse zurückgegeben werden. Um die Schnittstelle verwenden zu können, muss ein Objekt der Klasse`Retrofit` erstellt werden, das dann mit dieser verbunden wird. Dabei hilft das Entwurfsmuster Erbauer. Zudem ist es empfohlen den Dienst nach außen mit Hilfe des *Singleton* Entwurfmusters zur Verfügung zu stellen.

Ein Element, das aus moderner App-Entwicklung nicht wegzudenken ist, ist die **RecyclerView**. Mit dieser kann eine Liste an Objekten in einer Art *ScrollView* dargestellt werden. Das besondere ist, das jeder Eintrag ein eigens definiertes Layout besitzt, welches wiederverwendet werden kann. Wird beispielsweise nach unten gescrollt, wird das oberste Layout wieder unten angefügt und mit einem neuen Element gefüllt. Diese Funktionalität muss aber nicht implementiert werden, sondern wird automatisch mitgeliefert. Elemente, die erstellt werden müssen, sind das Layout eines Eintrags, sowie ein Adapter, der als Hilfs-Schnittstelle dient. Ein wichtiges Element davon ist ein öffentliches Feld, in dem eine Liste der Daten gespeichert wird. Werden die Daten verändert, muss `notifyDataSetChanged()` aufgerufen werden. Die Methode `getItemCount()`, die überschrieben werden muss, soll die Länge der Liste an Einträgen zurückgeben. Danach ist es notwendig zwei weitere Methoden zu überschreiben. `onBindViewHolder(`, die einen *ViewHolder*, der ein Container für ein Element darstellt, und die Position des Eintrags übergeben bekommt, verbindet die beiden Elemente. Vorher muss der *ViewHolder* jedoch in`onCreateViewHolder()` erzeugt werden. Diese Methode erzeugt und verbindet das Layout und gibt das erstellte Objekt zurück. Da das Layout eines Elements beliebig erstellt werden kann, muss ein eigener, passender *ViewHolder* definiert werden. Dafür wird eine innere Klasse verwendet, die ein *binding*-Objekt des Layouts übergeben bekommt. Mit der Methode`bind()`, die einen Eintrag übergeben bekommt, werden die Elemente des Layouts mit den passenden Daten gefüllt. Aufgerufen wird diese Methode in `onBindViewHolder()`. Zuletzt muss der Adapter der *RecyclerView* des Layouts verbunden werden. Dabei wird das Feld `adapter` auf ein Objekt des eigenen Adapters gesetzt. Anschließend müssen die Daten des Adapters, die in einem öffentlichen Feld vorliegen, gesetzt werden.

**Apps** können im *Google Play Store* mit Hilfe der *Google Play Console* **veröffentlicht** werden. Dazu benötigen Sie ein Entwicklerkonto. Bei der Registrierung fällt eine einmalige Gebühr von 25$ an. Der erste Schritte daraufhin ist die Vervollständigung der Entwickler- und Kontoinformationen. Erstere sind für den Storeeintrag wichtig.  
Eine Übersicht über die verknüpften Apps ist im Dashboard zu finden. Dort können Sie auch unter Angabe des Namen, sowie einigen weiteren Informationen, eine neue App erstellen. Daraufhin können Sie alle zugehörigen Informationen in der Übersicht der App hinterlegen. Ein wichtiger Teil ist hier zum Beispiel der Eintrag im *Play Store*.  
Veröffentlichen können Sie die App, indem Sie *Releases* der Produktion zuordnen. Dieser *Track* beschreibt die Versionshistorie der App im Store. Ein *Release* ist dabei eine signierte von *Android Studio* erstelle *APK* oder ein signiertes *App Bundle*. Dieses kann unter *Build* -> *Generate Signed Bundle/APK* erstellt werden. Wichtig ist, dass jede Version einen eindeutigen `versionCode` besitzt, der im *gradle.script* des Projekts angepasst werden kann. Der `versionName` hingegen kann auch öfter verwendet werden.  
Es stehen verschiedene Möglichkeiten zur Verfügung die App herauszugeben:
- Intern: Dieser Test ist besonders für Entwickler:innen interessant. Wird ein *Release* veröffentlicht, kann dieser direkt über einen Link heruntergeladen werden. Ein Update über den *Play Store* ist nicht möglich. Es ist zudem eine Liste an Tester:innen zu definieren.
- Alpha: In der Konsole wird diese Möglichkeit *Geschlossener Test* genannt. Der Release kann über den Play Store von einer definierten Gruppe an Tester:innen heruntergeladen und auch dort aktualisiert werden. Jede Version wird jedoch vorher von Google überprüft, was einige Zeit in Anspruch nehmen kann.
- Beta: Für einen Beta oder auch *Offenen Test* kann sich jede:r Play Store-Nutzer:in anmelden.
- Öffentlich: Geht ein *Release* in *Produktion*, ist er für alle im *Play Store* nach einer nochmaligen Überprüfung durch Google installierbar.

Soll die App im *Produktions-Track* erstmalig veröffentlicht werden, sind zudem weitere Informationen, wie eine Datenschutzerklärung, Angaben zur Werbung in der App, Zugang zu eingeschränkten Bereichen für die Überprüfer:innen, Zielgruppe, Inhalt und Kategorisierung notwendig. Zudem muss der Store-Eintrag vollständig sein.